In [1]:
import numpy as np
import skimage
import utils
import pathlib

In [2]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # You can also define other helper functions
    # Compute normalized histogram
    """
    1. Compute the normalized histogram of the input image. .denote the components of the histogram by p_i,i *= 0,1,2,...,L-1
    2. Compute the cumulative sums, P_1(k), for k = 0,1,2,...,L-1 
    3. Compute the cumulative means, m(k) for k = 0,1,2,...,L-1
    4. Compute the global mean, m_G
    5. Compute the between-class variance term, Sigma^2_B(k), for k = 0,1,2,...,L-1
    6. Obtain the Otsu threshold, k*, as the value of k for which Sigma^2_B(k) is maximum. 
        If the maximum is not unique, obtain k* by averaging the values of k corresponding to the various maxima detected. 
    # // 7.Compute the global variance Sigma^2_G, and then obtain the separability measure, n*
    """
    H, W = im.shape
    
    L = 256 # color values
    
    # histogram
    p, _ = np.histogram(im, bins=L, range=(0,L))
    
    # normalize
    p = p.astype(np.float64) / (H * W)
    
    P_1 = np.zeros(L)
    m = np.zeros(L)
    Sigma_square_B = np.zeros(L)
    
    # cumulative sums
    sum = 0.0
    for i in range(L):
        sum += p[i] 
        P_1[i] = sum
    
    # cumulative mean
    sum = 0.0
    for i in range(L):
        sum += i * p[i]
        m[i] = sum
    
    # global mean
    mG = m[len(m) - 1]
    
    # between-class variance
    for i in range(L):
        denominator = P_1[i] * (1 - P_1[i])
        
        if denominator <= 0:  # avoid division by zero
            Sigma_square_B[i] = 0
            continue
        
        numerator = (mG * P_1[i] - m[i]) ** 2
        Sigma_square_B[i] = numerator / denominator
    
    # otsu threshold
    kStar = np.argmax(Sigma_square_B)
    
    return kStar
    ### END YOUR CODE HERE ### 

In [3]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

        


Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed\rice-shaded-segmented.png
